## Dados sobre as reclamações realizadas no site:

- https://www.consumidor.gov.br/pages/principal/?1652964614719

- https://www.consumidor.gov.br/pages/dadosabertos/externo/

- Dados referentes ao ano de 2021 (Janeiro até Dezembro)
- Dados referentes ao ano de 2022 (Janeiro até Dezembro)
- Dados referentes ao ano de 2023 (Janeiro até Dezembro)
- Dados referentes ao ano de 2024 (Janeiro até Dezembro)
- Dados referentes ao ano de 2025 (Janeiro até Março)

### Realizando o Pre-Processamento dos Dados

### Bibliotecas Utilizadas:

In [2]:
# Imports

# Manipulaçao dos dados
import pandas as pd
import numpy as np

# Manipulação de arquivos/sistema
import glob
import os

# Ignore Warning(Avisos)
import warnings

warnings.filterwarnings("ignore")

pd.set_option('display.max_columns', 100)

### Carregando a base de dados:

In [3]:
# Unindo os dados dos meses de 2022(Janeiro Até Dezembro) - 2025(Até Janeiro).

# Criando uma variável com a extensão do arquivo a ser lido
extensao = 'csv'

# Agrupando/Concatenando os arqquivos, tudo em um único DataFrame
arquivos_csv = [i for i in glob.glob(f'../Data/Data_for_csv/*.{extensao}')]
df_gov = pd.concat([pd.read_csv(f, sep=';') for f in arquivos_csv])

### Ajustando o índice do dataset

In [4]:
# Verificando a quantidade de índices duplicados
print('Qtd. de índices duplicados:')
print(df_gov.index.duplicated().sum())

# Criando um novo índice
print('++++++')
df_gov.reset_index(inplace=True, drop=True)

# Verificando a quantidade de índices duplicados
print('Qtd. de índices duplicados:')
print(df_gov.index.duplicated().sum())

Qtd. de índices duplicados:
5278041
++++++
Qtd. de índices duplicados:
0


### Informações do dataset

In [5]:
# Verificando o nome das colunas do DataFrame
df_gov.columns

Index(['Região', 'UF', 'Cidade', 'Sexo', 'Faixa Etária', 'Data Finalização',
       'Tempo Resposta', 'Nome Fantasia', 'Segmento de Mercado', 'Área',
       'Assunto', 'Grupo Problema', 'Problema', 'Como Comprou Contratou',
       'Procurou Empresa', 'Respondida', 'Situação', 'Avaliação Reclamação',
       'Nota do Consumidor'],
      dtype='object')

In [6]:
# Verificando a quantidade de linhas e colunas do DataFrame
df_gov.shape

(5431641, 19)

In [7]:
# Info do dataset
df_gov.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5431641 entries, 0 to 5431640
Data columns (total 19 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   Região                  object 
 1   UF                      object 
 2   Cidade                  object 
 3   Sexo                    object 
 4   Faixa Etária            object 
 5   Data Finalização        object 
 6   Tempo Resposta          float64
 7   Nome Fantasia           object 
 8   Segmento de Mercado     object 
 9   Área                    object 
 10  Assunto                 object 
 11  Grupo Problema          object 
 12  Problema                object 
 13  Como Comprou Contratou  object 
 14  Procurou Empresa        object 
 15  Respondida              object 
 16  Situação                object 
 17  Avaliação Reclamação    object 
 18  Nota do Consumidor      float64
dtypes: float64(2), object(17)
memory usage: 787.4+ MB


In [8]:
# Descrição do dataset 
df_gov.describe()

,Tempo Resposta,Nota do Consumidor
count,5.328916e+06,2.691130e+06
mean,6.668791e+00,2.855284e+00
std,4.519879e+02,1.794015e+00
min,-7.368760e+05,1.000000e+00
25%,4.000000e+00,1.000000e+00
50%,7.000000e+00,3.000000e+00
75%,9.000000e+00,5.000000e+00
max,7.386540e+05,5.000000e+00


### Verificando valores ausentes

In [9]:
# Calcula o total de células no dataset
totalCells = np.prod(df_gov.shape)

# Conta o número de valores ausentes por coluna
missingCount = df_gov.isnull().sum()

# Calcula o total de valores ausentes
totalMissing = missingCount.sum()

# Calcula o percentual de valores ausentes
print("O Dataset tem", round(((totalMissing/totalCells) * 100), 2), "%", "de valores ausentes.")

O Dataset tem 2.76 % de valores ausentes.


In [10]:
# Total de valores ausentes
mis_val = df_gov.isnull().sum()

# Porcentagem de valores ausentes
mis_val_percent = 100 * mis_val / len(df_gov)

# Tipo de dado das colunas com valores ausentes
mis_val_dtype = df_gov.dtypes

# Cria uma tabela com os resultados
mis_val_table = pd.concat([mis_val, mis_val_percent, mis_val_dtype], axis=1)

# Renomear as colunas
mis_val_table_ren_columns = mis_val_table.rename(
columns = {0 : 'Valores Ausentes', 1 : '% de Valores Ausentes', 2: 'Dtype'})

# Classifica a tabela por porcentagem de valores ausentes de forma decrescente e remove colunas sem valores faltantes
mis_val_table_ren_columns = mis_val_table_ren_columns[mis_val_table_ren_columns.iloc[:,0] != 0].sort_values('% de Valores Ausentes', ascending = False).round(2)

# Print 
print ("O dataset tem " + str(df_gov.shape[1]) + " colunas.\n"
        "Encontrado: " + str(mis_val_table_ren_columns.shape[0]) + " colunas que têm valores ausentes.")

# Retorna o dataframe com informações ausentes
mis_val_table_ren_columns

O dataset tem 19 colunas.
Encontrado: 4 colunas que têm valores ausentes.


,Valores Ausentes,% de Valores Ausentes,Dtype
Nota do Consumidor,2740511,50.45,float64
Tempo Resposta,102725,1.89,float64
Sexo,322,0.01,object
Avaliação Reclamação,6,0.00,object


> Vamos adotar o seguinte procedimento para cada coluna:

- Coluna 'Nota do Consumidor' -> Será imputado valores utilizando o método de preenchimento progressivo (forward fill).
- Coluna 'Tempo Resposta' -> será imputado o valor 0(Zero) para os valores NaN - Pois são reclamações que não foram respondidas.
- Coluna 'Sexo' -> será imputado o valor 'O'(Outros) para os valores NaN
- Coluna 'Avaliação Reclamação' -> Dados Ausentes seram removidos

In [11]:
# Imputando na coluna 'Nota do Consumidor' valores utilizando o método (forward fill)
df_gov['Nota do Consumidor'] = df_gov['Nota do Consumidor'].fillna(method='ffill')

# Imputando na coluna 'Tempo Resposta' o valor 0(Zero) nos valores NaN
df_gov['Tempo Resposta'].fillna(0, inplace=True)

# Imputando na coluna 'Sexo' o Valor O(Outros) nos Valores NaN
df_gov['Sexo'].fillna('O', inplace=True)

# Eliminando valores ausentes na coluna 'Avaliação Reclamação'
df_gov.dropna(inplace=True)

In [12]:
# Calcula o total de células no dataset
totalCells = np.prod(df_gov.shape)

# Conta o número de valores ausentes por coluna
missingCount = df_gov.isnull().sum()

# Calcula o total de valores ausentes
totalMissing = missingCount.sum()

# Calcula o percentual de valores ausentes
print("O Dataset tem", round(((totalMissing/totalCells) * 100), 2), "%", "de valores ausentes.")

O Dataset tem 0.0 % de valores ausentes.


> Dataset agora sem valores ausentes

### Verificando/Eliminando linhas duplicadas

In [13]:
# Conta a quantidade de linhas do dataframe
old = df_gov.shape[0]

# Elimina/apaga as linhas duplicadas
df_gov.drop_duplicates(inplace = True)

# Conta a quantidade de linhas do dataframe (Novo dataframe)
new = df_gov.shape[0]

# Verifica a quandidade de linha duplicadas
count = old - new
if (count == 0):
    print("Nenhuma linha duplicada foi encontrada.")
else:
    print(f"{count} linhas duplicadas foram encontradas e removidas.")

19408 linhas duplicadas foram encontradas e removidas.


In [14]:
# Verificando a quantidade de linhas e colunas do DataFrame
df_gov.shape

(5412204, 19)

### Verificando valores únicos

- Coluna 'Região'

In [15]:
df_gov['Região'].unique()

array(['NE', 'S ', 'SE', 'N ', 'CO'], dtype=object)

- Coluna 'UF'

In [16]:
df_gov['UF'].unique()

array(['CE', 'RS', 'SP', 'RJ', 'MA', 'PA', 'PE', 'DF', 'PR', 'BA', 'SE',
       'PI', 'TO', 'MS', 'RO', 'AM', 'RN', 'AL', 'PB', 'MG', 'SC', 'AC',
       'ES', 'GO', 'MT', 'AP', 'RR'], dtype=object)

- Coluna 'Sexo'

In [17]:
df_gov['Sexo'].unique()

array(['M', 'F', 'O'], dtype=object)

- Coluna 'Procurou Empresa'

In [18]:
df_gov['Procurou Empresa'].unique()

array(['S', 'N'], dtype=object)

- Coluna 'Respondida'

In [19]:
df_gov['Respondida'].unique()

array(['N', 'S'], dtype=object)

- Coluna 'Situação'

In [20]:
df_gov['Situação'].unique()

array(['Finalizada avaliada', 'Finalizada não avaliada'], dtype=object)

### Criando novas variáveis

In [21]:
# Criando a variavel 'Solicitação' aonde cada linha vai receber o numero 1 como identificação
df_gov['Solicitação'] = 1

In [22]:
# Transformando a coluna 'Data Finalização' de object para datetime
df_gov['Data Finalização'] = pd.to_datetime(df_gov['Data Finalização'], format='mixed')

# Informações do tipo dos dados das colunas
df_gov.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5412204 entries, 23 to 5431640
Data columns (total 20 columns):
 #   Column                  Dtype         
---  ------                  -----         
 0   Região                  object        
 1   UF                      object        
 2   Cidade                  object        
 3   Sexo                    object        
 4   Faixa Etária            object        
 5   Data Finalização        datetime64[ns]
 6   Tempo Resposta          float64       
 7   Nome Fantasia           object        
 8   Segmento de Mercado     object        
 9   Área                    object        
 10  Assunto                 object        
 11  Grupo Problema          object        
 12  Problema                object        
 13  Como Comprou Contratou  object        
 14  Procurou Empresa        object        
 15  Respondida              object        
 16  Situação                object        
 17  Avaliação Reclamação    object        
 18  Nota d

In [23]:
# Criando a variável Mes(Número do mês)
df_gov['Mes Finalização'] = df_gov['Data Finalização'].dt.month

# Criando a variável Mes Nome(Com o nome do mês)
df_gov['Mes Nome Finalização'] = 'NaN'

df_gov.loc[df_gov['Mes Finalização'] == 1, 'Mes Nome Finalização'] = 'Janeiro'
df_gov.loc[df_gov['Mes Finalização'] == 2, 'Mes Nome Finalização'] = 'Fevereiro'
df_gov.loc[df_gov['Mes Finalização'] == 3, 'Mes Nome Finalização'] = 'Março'
df_gov.loc[df_gov['Mes Finalização'] == 4, 'Mes Nome Finalização'] = 'Abril'
df_gov.loc[df_gov['Mes Finalização'] == 5, 'Mes Nome Finalização'] = 'Maio'
df_gov.loc[df_gov['Mes Finalização'] == 6, 'Mes Nome Finalização'] = 'Junho'
df_gov.loc[df_gov['Mes Finalização'] == 7, 'Mes Nome Finalização'] = 'Julho'
df_gov.loc[df_gov['Mes Finalização'] == 8, 'Mes Nome Finalização'] = 'Agosto'
df_gov.loc[df_gov['Mes Finalização'] == 9, 'Mes Nome Finalização'] = 'Setembro'
df_gov.loc[df_gov['Mes Finalização'] == 10, 'Mes Nome Finalização'] = 'Outubro'
df_gov.loc[df_gov['Mes Finalização'] == 11, 'Mes Nome Finalização'] = 'Novembro'
df_gov.loc[df_gov['Mes Finalização'] == 12, 'Mes Nome Finalização'] = 'Dezembro'

# Verificando os valores únicos
df_gov['Mes Nome Finalização'].unique()

array(['Janeiro', 'Fevereiro', 'Março', 'Abril', 'Maio', 'Junho', 'Julho',
       'Agosto', 'Setembro', 'Outubro', 'Novembro', 'Dezembro'],
      dtype=object)

In [24]:
df_gov.tail(5)

,Região,UF,Cidade,Sexo,Faixa Etária,Data Finalização,Tempo Resposta,Nome Fantasia,Segmento de Mercado,Área,Assunto,Grupo Problema,Problema,Como Comprou Contratou,Procurou Empresa,Respondida,Situação,Avaliação Reclamação,Nota do Consumidor,Solicitação,Mes Finalização,Mes Nome Finalização
5431636,NE,MA,São Luís,M,entre 31 a 40 anos,2025-03-31,0.0,Equatorial Maranhão (CEMAR),Energia Elétrica,"Água, Energia, Gás",Energia Elétrica,Atendimento / SAC,Falta de acessibilidade ou de atendimento prio...,Domicílio,N,S,Finalizada avaliada,Resolvida,5.0,1,3,Março
5431637,NE,PI,Jardim do Mulato,M,entre 51 a 60 anos,2025-03-31,0.0,Equatorial Piauí,Energia Elétrica,"Água, Energia, Gás",Energia Elétrica,Atendimento / SAC,SAC - Dificuldade de contato / acesso,Internet,S,S,Finalizada avaliada,Resolvida,5.0,1,3,Março
5431638,NE,MA,São Luís,M,entre 31 a 40 anos,2025-03-31,0.0,Equatorial Maranhão (CEMAR),Energia Elétrica,"Água, Energia, Gás",Energia Elétrica,Atendimento / SAC,Dificuldade de contato / acesso a outros canai...,Internet,N,S,Finalizada avaliada,Resolvida,5.0,1,3,Março
5431639,NE,MA,São Luís,M,entre 31 a 40 anos,2025-03-31,0.0,Equatorial Maranhão (CEMAR),Energia Elétrica,"Água, Energia, Gás",Energia Elétrica,Atendimento / SAC,Dificuldade de contato / acesso a outros canai...,Domicílio,N,S,Finalizada avaliada,Resolvida,5.0,1,3,Março
5431640,SE,SP,São Paulo,M,entre 31 a 40 anos,2025-03-31,0.0,Concessionária CCR RioSP,Transporte Terrestre,Transportes,Rodovias Concedidas / Pedagiadas,Atendimento / SAC,SAC - Demanda não resolvida / não respondida /...,Internet,S,S,Finalizada avaliada,Não Resolvida,1.0,1,3,Março


In [25]:
# Criando a variável Dia da Semana(Número do Dia da Semana)
df_gov['Dia Semana Finalização'] = df_gov['Data Finalização'].dt.weekday

# Criando a variável Dia Semana Nome Finalização(Com o nome do dia da semana)
df_gov['Dia Semana Nome Finalização'] = 'NaN'

df_gov.loc[df_gov['Dia Semana Finalização'] == 0, 'Dia Semana Nome Finalização'] = 'Segunda-Feira'
df_gov.loc[df_gov['Dia Semana Finalização'] == 1, 'Dia Semana Nome Finalização'] = 'Terça-Feira'
df_gov.loc[df_gov['Dia Semana Finalização'] == 2, 'Dia Semana Nome Finalização'] = 'Quarta-Feira'
df_gov.loc[df_gov['Dia Semana Finalização'] == 3, 'Dia Semana Nome Finalização'] = 'Quinta-Feira'
df_gov.loc[df_gov['Dia Semana Finalização'] == 4, 'Dia Semana Nome Finalização'] = 'Sexta-Feira'
df_gov.loc[df_gov['Dia Semana Finalização'] == 5, 'Dia Semana Nome Finalização'] = 'Sábado'
df_gov.loc[df_gov['Dia Semana Finalização'] == 6, 'Dia Semana Nome Finalização'] = 'Domingo'

# Verificando os valores únicos
df_gov['Dia Semana Nome Finalização'].unique()

array(['Terça-Feira', 'Segunda-Feira', 'Quarta-Feira', 'Sexta-Feira',
       'Sábado', 'Quinta-Feira', 'Domingo'], dtype=object)

In [26]:
# Criando a variável Ano
df_gov['Ano'] = df_gov['Data Finalização'].dt.year

# Verificando os valores únicos
df_gov['Ano'].unique()

array([2021, 2022, 2023, 2024, 2025], dtype=int32)

### Renomeando Nome das Colunas

In [27]:
df_gov.rename(columns={
    'Região': 'Regiao',
    'Faixa Etária': 'Faixa_Etaria',
    'Data Finalização': 'Data_Finalizacao',
    'Tempo Resposta': 'Tempo_Resposta',
    'Nome Fantasia': 'Nome_Fantasia',
    'Segmento de Mercado': 'Segmento_de_Mercado',
    'Área': 'Area',
    'Grupo Problema': 'Grupo_Problema',
    'Como Comprou Contratou': 'Como_Comprou_Contratou',
    'Procurou Empresa': 'Procurou_Empresa',
    'Situação': 'Situacao',
    'Avaliação Reclamação': 'Avaliacao_Reclamacao',
    'Nota do Consumidor': 'Nota_do_Consumidor',
    'Solicitação': 'Solicitacao',
    'Mes Finalização': 'Mes_Finalizacao',
    'Mes Nome Finalização': 'Mes_Nome_Finalizacao',
    'Dia Semana Finalização': 'Dia_Semana_Finalizacao',
    'Dia Semana Nome Finalização': 'Dia_Semana_Nome_Finalizacao'}, inplace=True)

In [28]:
# Amostra
df_gov.head()

,Regiao,UF,Cidade,Sexo,Faixa_Etaria,Data_Finalizacao,Tempo_Resposta,Nome_Fantasia,Segmento_de_Mercado,Area,Assunto,Grupo_Problema,Problema,Como_Comprou_Contratou,Procurou_Empresa,Respondida,Situacao,Avaliacao_Reclamacao,Nota_do_Consumidor,Solicitacao,Mes_Finalizacao,Mes_Nome_Finalizacao,Dia_Semana_Finalizacao,Dia_Semana_Nome_Finalizacao,Ano
23,NE,CE,Fortaleza,M,entre 31 a 40 anos,2021-01-05,0.0,Faculdade Unopar,Estabelecimentos de Ensino,Educação,Superior ( Graduação e Pós ),Atendimento / SAC,Má qualidade no atendimento (descortesia / des...,Loja física,S,N,Finalizada avaliada,Não Resolvida,1.0,1,1,Janeiro,1,Terça-Feira,2021
24,S,RS,Horizontina,F,mais de 70 anos,2021-01-04,15.0,Itaú Consignado,"Bancos, Financeiras e Administradoras de Cartão",Serviços Financeiros,Crédito Consignado / Cartão de Crédito Consign...,Cobrança / Contestação,Cobrança por serviço/produto não contratado / ...,Não comprei / contratei,N,S,Finalizada não avaliada,Não Avaliada,1.0,1,1,Janeiro,0,Segunda-Feira,2021
25,SE,SP,Caieiras,M,entre 21 a 30 anos,2021-01-06,15.0,Itauleasing,"Bancos, Financeiras e Administradoras de Cartão",Serviços Financeiros,Banco de Dados e Cadastros de Consumidores (SP...,Informação,"Dados pessoais ou financeiros consultados, col...",Não comprei / contratei,S,S,Finalizada não avaliada,Não Avaliada,1.0,1,1,Janeiro,2,Quarta-Feira,2021
26,SE,SP,São Paulo,M,entre 21 a 30 anos,2021-01-01,13.0,Banco Santander Cartões,"Bancos, Financeiras e Administradoras de Cartão",Serviços Financeiros,Cartão de Crédito / Cartão de Débito / Cartão ...,Cobrança / Contestação,Cobrança por serviço/produto não contratado / ...,Não comprei / contratei,S,S,Finalizada não avaliada,Não Avaliada,1.0,1,1,Janeiro,4,Sexta-Feira,2021
27,SE,RJ,Rio de Janeiro,F,entre 31 a 40 anos,2021-01-02,15.0,Itaucard,"Bancos, Financeiras e Administradoras de Cartão",Demais Serviços,"Programas de Fidelidade / Benefícios (pontos, ...",Contrato / Oferta,Oferta não cumprida / serviço não fornecido/ v...,Não comprei / contratei,S,S,Finalizada não avaliada,Não Avaliada,1.0,1,1,Janeiro,5,Sábado,2021


### Salvando dataset em um arquivo .csv

In [29]:
# Salvando na pasta 'Data'

import sqlite3

df_gov.to_csv('../Data/Data_for_clear/dfGov.csv', sep=';', index=False)
df_gov.to_parquet('../Data/Data_for_clear/GOLDEN_dfGov.parquet', index=False)

conn = sqlite3.connect('../Data/Data_for_clear/dfGov.db')

df_gov.to_sql('dfGov', conn, if_exists='replace', index=False)

5412204